In [112]:
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile
import sounddevice as sd
import soundfile as sf

from Code.audio import AudioPreproc, AudioPreprocConfig, AudioFeat, AudioFeatConfig
from Code.audio.Standardizer import Standardizer
from Code.audio import AudioOrchestrator


def infer_label_from_name(p: Path) -> str:
    name = p.stem.lower()  # 'contar001' -> 'contar001'
    if name.startswith("contar"):
        return "contar"
    elif name.startswith("proporcion"):
        return "proporcion"
    elif name.startswith("salir"):
        return "salir"
    else:
        raise ValueError(f"No sé qué clase es '{p.name}'")

def grabar_audio_segundos(segundos: float = 2.0) -> tuple[np.ndarray, int]:
    dev = sd.query_devices(kind='input')
    sr_rec = int(dev['default_samplerate'] or 16000)
    print(f"Grabando {segundos}s a {sr_rec} Hz (input device default)…")
    audio = sd.rec(int(segundos * sr_rec), samplerate=sr_rec, channels=1, dtype="float32")
    sd.wait()
    return audio.squeeze(), sr_rec

run = 2

input_dir = PROJECT_ROOT / "Database" / "data" / "audio"
output_dir = PROJECT_ROOT / "Database" / "tmp" / "audio" / f"audioTry{run:02d}"
output_dir.mkdir(parents=True, exist_ok=True)

model_number = 1
model_path = PROJECT_ROOT / "Database" / "models" / "audio" / f"modelo_audio_{model_number}.npz"

orch = AudioOrchestrator()

paths = sorted(input_dir.rglob("*.wav"))
labels = [p.parent.name.lower() for p in paths]

R_dic = orch.entrenar(paths=paths, labels=labels)
orch.guardar_modelo(model_path)


display(R_dic)

y, sr = grabar_audio_segundos()

resultado = orch.predecir_comando((y, sr), devolver_distancia=True)

print("Comando predicho:", resultado["label"])
print("Distancia mínima:", resultado["distancia_min"])
print("\n")

Xproj = orch._X_store_proj
labs = orch._y_store
preds = []
for i, (x, lab) in enumerate(zip(Xproj, labs)):
    pred = orch.knn.predecir(x, exclude_idx=i)
    preds.append(pred)
    if pred != lab:
        print("Falla en train idx", i, "real", lab, "pred", pred)
print("\nAcc LOO:", sum(p==l for p,l in zip(preds,labs)) / len(labs))

d = orch.knn.distancias(orch.stats.transform_one(orch.feat.extraer_caracteristicas(*orch._preprocesar_audio((y,sr)))) @ orch._eigvecs[:, :orch._k_used])
print("dist min:", d.min(), "dist medias por clase:")
import numpy as np
for c in set(labs):
    mask = labs == c
    print(c, d[mask].mean())



{'N': 33, 'D_raw': 168, 'D_proj': 22, 'var_retenida': 0.9548174186745153}

Grabando 2.0s a 44100 Hz (input device default)…
Comando predicho: proporcion
Distancia mínima: 0.4893041253089905


Falla en train idx 2 real contar pred proporcion
Falla en train idx 4 real contar pred proporcion
Falla en train idx 5 real contar pred proporcion
Falla en train idx 8 real contar pred proporcion
Falla en train idx 11 real proporcion pred salir
Falla en train idx 18 real proporcion pred contar
Falla en train idx 28 real salir pred contar
Falla en train idx 29 real salir pred contar

Acc LOO: 0.7575757575757576
dist min: 0.48930413 dist medias por clase:
proporcion 0.9324585
contar 1.0401521
salir 1.0597081
